# Install requirements and Imports
1. Run the below cell in order to **correct install all the libraries needed**, and wait for its completion (see the comment below)

In [ ]:
################################################################################
##### WAIT FOR COMPLETE EXECUTION OF THIS CELL BEFORE RUNNING THE FOLLOWING ONES
################################################################################
%%capture
! pip uninstall --yes numpy
! pip install numpy==1.19.5

import os
os.kill(os.getpid(), 9)

2. Run this cell, **after the completion of the one above it**

In [1]:
%%capture
! pip install tensorflow==2.4.1
! pip install nengo==3.1.0
! pip install nengo-dl==3.4.0

3. Import all the **libraries**

In [2]:
import nengo
import tensorflow as tf
import numpy as np
import nengo_dl
import pickle
from nengo.utils.filter_design import cont2discrete
from urllib.request import urlretrieve

# Classification (LMU)
LMUCell used to **perform the simulation**

In [3]:
class LMUCell(nengo.Network):
    def __init__(self, units, order, theta, input_d, **kwargs):
        super().__init__(**kwargs)

        Q = np.arange(order, dtype=np.float64)
        R = (2 * Q + 1)[:, None] / theta
        j, i = np.meshgrid(Q, Q)

        A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R
        B = (-1.0) ** Q[:, None] * R
        C = np.ones((1, order))
        D = np.zeros((1,))

        A, B, _, _, _ = cont2discrete((A, B, C, D), dt=1.0, method="zoh")

        with self:
            nengo_dl.configure_settings(trainable=None)

            self.x = nengo.Node(size_in=input_d)
            self.u = nengo.Node(size_in=1)
            self.m = nengo.Node(size_in=order)
            self.h = nengo_dl.TensorNode(tf.nn.tanh, shape_in=(units,), pass_time=False)

            nengo.Connection(
                self.x, self.u, transform=np.ones((1, input_d)), synapse=None
            )

            conn_A = nengo.Connection(self.m, self.m, transform=A, synapse=0)
            self.config[conn_A].trainable = False
            conn_B = nengo.Connection(self.u, self.m, transform=B, synapse=None)
            self.config[conn_B].trainable = False

            nengo.Connection(
                self.x, self.h, transform=nengo_dl.dists.Glorot(), synapse=None
            )
            nengo.Connection(
                self.h, self.h, transform=nengo_dl.dists.Glorot(), synapse=0
            )
            nengo.Connection(
                self.m,
                self.h,
                transform=nengo_dl.dists.Glorot(),
                synapse=None,
            )

**Download** the *Cluster Separation* in order to perfrorm the application of the LMU over the clusters

In [4]:
print("Download the clusters separation: \n")
!gdown --fuzzy "https://drive.google.com/file/d/1iCKL4e-IPXnyFxjsWOB3ILqG72UBst-i/view?usp=sharing"
separate_datasets = pickle.load(open("separate_datasets.pkl", "rb"))

Download the clusters separation: 

Downloading...
From: https://drive.google.com/uc?id=1iCKL4e-IPXnyFxjsWOB3ILqG72UBst-i
To: /content/separate_datasets.pkl
100% 375M/375M [00:05<00:00, 63.6MB/s]


Create **all the networks related to different clusters**

In [5]:
seed = 0
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

nets = []

for clst in separate_datasets.keys():
  with nengo.Network(seed=seed) as net:
      nengo_dl.configure_settings(
          trainable=None,
          stateful=False,
          keep_history=False,
      )

      inp = nengo.Node(np.zeros(separate_datasets[clst]['transformed'].shape[-1]))

      lmu = LMUCell(
          units=212,
          order=256,
          theta=separate_datasets[clst]['transformed'].shape[1],
          input_d=separate_datasets[clst]['transformed'].shape[-1],
      )

      conn = nengo.Connection(inp, lmu.x, synapse=None)
      net.config[conn].trainable = False

      out = nengo.Node(size_in=len(separate_datasets[clst]['classes']))

      nengo.Connection(lmu.h, out, transform=nengo_dl.dists.Glorot(), synapse=None)

      p = nengo.Probe(out)
  nets.append(net)

Perform the **simulation for all the networks**

In [12]:
do_training = False

ids = {0: ["lmu_params0", "1uotAmrALQUD83i45caVGPcuNGaXzuy2H"],
       1: ["lmu_params1", "1AvQ8U6cXdsHTqAGk-jxtPXcfM8XTis5x"],
       2: ["lmu_params2", "1lPVx00fGxCyON4F3S3ozwVEY3UOkiKwa"],
       3: ["lmu_params3", "1WuCr6vRO8aNuerh2znjpWefocz7wIebM"]}

train_dict = {}  #{0: {'data': data, 'label': [0, 1, ... ]}, ...}
test_dict = {} #{0: {'data': data, 'label': [0, 1, ... ]}, ...}

for (clst, net) in enumerate(nets):
  train_dict[clst] = {}
  test_dict[clst] = {}

  test_dict[clst]['data'] = separate_datasets[clst]['transformed_test']
  train_dict[clst]['data'] = separate_datasets[clst]['transformed']

  test_dict[clst]['labels'] = []
  train_dict[clst]['labels'] = []

  for label in separate_datasets[clst]['labels']:
    train_dict[clst]['labels'].append(np.where(separate_datasets[clst]['classes'] == label)[0][0])

  for label in separate_datasets[clst]['test_labels']:
    index = np.where(separate_datasets[clst]['classes'] == label)[0]
    test_dict[clst]['labels'].append(index[0])

  test_dict[clst]['labels'] = np.array(test_dict[clst]['labels'])[:, None, None]
  train_dict[clst]['labels'] = np.array(train_dict[clst]['labels'])[:, None, None]

  with nengo_dl.Simulator(net, minibatch_size=100 #, unroll_simulation=train_dict[clst]['unroll']
                          ) as sim:
      sim.compile(
          loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
          optimizer=tf.optimizers.Adam(),
          metrics=["accuracy"],
      )

      test_acc = sim.evaluate(test_dict[clst]['data'], test_dict[clst]['labels'], verbose=1)["probe_accuracy"]
      print(f"Initial test accuracy: {test_acc * 100:.2f}%")

      if do_training:
          sim.fit(train_dict[clst]['data'], train_dict[clst]['labels'], epochs=10)
          sim.save_params("./lmu_params"+str(clst))
          print("\n")
      else:
          urlretrieve(
              "https://drive.google.com/uc?export=download&"
              "id="+ids[clst][1],
              ids[clst][0]+".npz",
          )
          sim.load_params(ids[clst][0])

      test_acc = sim.evaluate(test_dict[clst]['data'], test_dict[clst]['labels'], verbose=1)["probe_accuracy"]
      print(f"Final test accuracy: {test_acc * 100:.2f}%\n")

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
|           Constructing graph: pre-build stage (0%)           | ETA:  --:--:--

/usr/local/lib/python3.7/dist-packages/nengo_dl/simulator.py:461: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  "No GPU support detected. See "


Construction finished in 0:00:01                                               
|###           Constructing graph: build stage (5%)              | ETA: 0:00:01

/usr/local/lib/python3.7/dist-packages/nengo_dl/simulator.py:1930: UserWarning: Number of elements in input data (6328) is not evenly divisible by Simulator.minibatch_size (100); input data will be truncated.
  % (data_batch, self.minibatch_size)


63/63 [==============================] - 6s 74ms/step - loss: 1.2195 - probe_loss: 1.2195 - probe_accuracy: 0.3685
Initial test accuracy: 36.71%
63/63 [==============================] - 5s 74ms/step - loss: 0.4511 - probe_loss: 0.4511 - probe_accuracy: 0.8292
Final test accuracy: 82.92%

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
|###           Constructing graph: build stage (5%)              | ETA: 0:00:01

...

101/101 [==============================] - 109s 1s/step - loss: 1.9349 - probe_loss: 1.9349 - probe_accuracy: 0.1996
Initial test accuracy: 20.68%
101/101 [==============================] - 106s 1s/step - loss: 0.5516 - probe_loss: 0.5516 - probe_accuracy: 0.8118
Final test accuracy: 81.18%

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
|###           Constructing graph: build stage (5%)              | ETA: 0:00:01

/usr/local/lib/python3.7/dist-packages/nengo_dl/simulator.py:1930: UserWarning: Number of elements in input data (8288) is not evenly divisible by Simulator.minibatch_size (100); input data will be truncated.
  % (data_batch, self.minibatch_size)


82/82 [==============================] - 73s 874ms/step - loss: 1.7203 - probe_loss: 1.7203 - probe_accuracy: 0.2389
Initial test accuracy: 23.76%
82/82 [==============================] - 73s 891ms/step - loss: 0.5248 - probe_loss: 0.5248 - probe_accuracy: 0.7962
Final test accuracy: 79.62%

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
|###           Constructing graph: build stage (5%)              | ETA: 0:00:01

/usr/local/lib/python3.7/dist-packages/nengo_dl/simulator.py:1930: UserWarning: Number of elements in input data (12144) is not evenly divisible by Simulator.minibatch_size (100); input data will be truncated.
  % (data_batch, self.minibatch_size)


121/121 [==============================] - 175s 1s/step - loss: 2.2444 - probe_loss: 2.2444 - probe_accuracy: 0.1650
Initial test accuracy: 16.53%
121/121 [==============================] - 174s 1s/step - loss: 0.8170 - probe_loss: 0.8170 - probe_accuracy: 0.7387
Final test accuracy: 73.87%

